## set running directory to project source

In [1]:
import os
import sys
sys.path.append(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

from pkg.utils.code_scraps import add_indy_sweep_tool, \
        use_current_place_point_only, use_current_sub_binders_only, \
        finish_L_shape, set_l_shape_object, ModeSwitcher, double_sweep_motions

from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder
from pkg.planning.scene import PlanningScene
from pkg.planning.pipeline import PlanningPipeline
from pkg.ui.ui_broker import *
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepFramer, FixtureSlot
from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask, AbstractObject
from pkg.planning.constraint.constraint_subject import SweepLineTask
from pkg.planning.constraint.constraint_subject import SweepFrame
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.task.rrt import TaskRRT
from pkg.planning.constraint.constraint_common import sample_redundancy, combine_redundancy
from pkg.planning.sampling.node_sampling import make_state_param_hashable, UniformNodeSampler, PenaltyNodeSampler, GrowingSampler
from pkg.planning.task.custom_rules.sweep_entrance_control import SweepEntranceControlRule
import matplotlib.pyplot as plt
import math
import numpy as np
from collections import defaultdict
from demo_utils.area_select import DATASET_DIR, SweepDirections
try_mkdir(DATASET_DIR)

## create scene builder

In [2]:
ROBOT_TYPE = RobotType.indy7
INDY_BASE_OFFSET = (0.172,0,0.439)
TIMEOUT_MP = 0.5

mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                None)
robot_config = RobotConfig(1, ROBOT_TYPE, (INDY_BASE_OFFSET, (0,0,0)),
                INDY_IP, root_on="kmb0_platform", 
                           specs={"no_sdk":True})

crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config], connection_list=[False, False])

ROBOT_NAME = robot_config.get_indexed_name()
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")

connection command:
kmb0: False
indy1: False


## make geometry scene

In [3]:
BASE_LINK = "base_link"
EE_NAME = "ee_point"
gscene = s_builder.create_gscene(crob, start_rviz=True)

floor = gscene.create_safe(GEOTYPE.BOX, "floor", BASE_LINK, (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=False)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
# ee_point.draw_traj_coords([[0]*6])

Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


## init planning pipeline

In [4]:
from demo_utils.environment import add_kiro_indytool_down

pscene = PlanningScene(gscene, combined_robot=crob)
TIP_LINK = pscene.robot_chain_dict[ROBOT_NAME]["tip_link"]
ROBOT_BASE = pscene.robot_chain_dict[ROBOT_NAME]['link_names'][0]
add_kiro_indytool_down(gscene, zoff=0, tool_link=TIP_LINK, face_name="dummy_brush")
ee_point = gscene.create_safe(GEOTYPE.SPHERE, EE_NAME, TIP_LINK, (0.01,)*3, 
                              center=(0,0,0), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.8), display=True, fixed=True, collision=False)

ppline = PlanningPipeline(pscene)

mplan = MoveitPlanner(pscene, enable_dual=False, incremental_constraint_motion=True)
mplan.update_gscene()
ppline.set_motion_planner(mplan)


# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)


## Set Parameters

In [5]:
TIMEOUT_MP = 0.5


VISUALIZE = False
VERBOSE = False
N_try = 3
SWEEP_DIV = 7


HOME_POSE_EXT = np.array([0,]*6 + np.deg2rad([  0., 50.,  -70.,  -0.,  -90., 0]).tolist())
crob.home_pose = HOME_POSE_EXT
crob.home_dict = list2dict(crob.home_pose, gscene.joint_names)
gscene.show_pose(crob.home_pose)

# # dummy tool coord
# gscene.add_highlight_axis("tool", "coord", link_name="indy0_tcp", 
#                           center=(0.1,0,0.2), orientation_mat=Rot_rpy((0,0,np.pi)))

 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


## Scan

In [ ]:
# HOME_POSE = np.zeros(gscene.joint_num)
HOME_POSE = HOME_POSE_EXT
for tip_dir, SWEEP_AXIS in [(SweepDirections.front, "Z"), (SweepDirections.front, "X"), (SweepDirections.up, "Z"), (SweepDirections.down, "Z")]:
    SWEEP_AX = "XYZ".index(SWEEP_AXIS)
    STEP_SIZES = [0.02]*3
    STEP_SIZES[SWEEP_AX] = 0.01
    X_step, Y_step, Z_step = STEP_SIZES

    if tip_dir == SweepDirections.front:
        X_range = (0.0, 1)
        Y_range = (-0.8, 0.8)
        Z_range = (0.0, 1.0)
    elif tip_dir == SweepDirections.up:
        X_range = (0.2, 0.8)
        Y_range = (-0.8, 0.8)
        Z_range = (-0.1, 1.3)
    elif tip_dir == SweepDirections.down:
        X_range = (0.2, 0.8)
        Y_range = (-0.8, 0.8)
        Z_range = (-0.7, 0.7)
    else:
        raise(NotImplementedError("SweepDirection not implemented"))
        
    DATASET_PATH = os.path.join(DATASET_DIR, SweepDirections.get_file_name(ROBOT_TYPE, tip_dir.name+SWEEP_AXIS))
    R_be, STEP_AX, LEVEL_AX = SweepDirections.get_Re_step_level(tip_dir, SWEEP_AX)
        
    gtimer = GlobalTimer.instance()
    gtimer.reset()

    X_DIR, Y_DIR, Z_DIR = np.identity(3, dtype=np.int)
    SWEEP_DIR = [X_DIR, Y_DIR, Z_DIR][SWEEP_AX]
    RANGE_STEPS_REF = [(X_range, X_step, X_DIR), (Y_range, Y_step, Y_DIR), (Z_range, Z_step, Z_DIR)]
    RANGEX, STEPX, DIRX = RANGE_STEPS_REF[SWEEP_AX]
    STEP_SWEEP = float(RANGEX[1] - RANGEX[0])/SWEEP_DIV
    
    RANGE1, STEP1, DIR1 = RANGE_STEPS_REF[LEVEL_AX]
    RANGE2, STEP2, DIR2 = RANGE_STEPS_REF[STEP_AX]
        

    range_list_dict = {}
    config_list_dict = {}
    STEPS_1 = np.round(np.arange(RANGE1[0], RANGE1[1]+STEP1, STEP1), 4)
    STEPS_2 = np.round(np.arange(RANGE2[0], RANGE2[1]+STEP2, STEP2), 4)
    STEPS_X = np.round(np.arange(RANGEX[0], RANGEX[1]-1e-3, STEP_SWEEP)[1:].tolist(), 4)
    LEN_1, LEN_2 = len(STEPS_1), len(STEPS_2)
    count = 0
    LEN_ALL = LEN_1*LEN_2
    gtimer.tic("test")
    for i_p1, pos1 in enumerate(STEPS_1):
        for i_p2, pos2 in enumerate(STEPS_2):
            count += 1
            with gtimer.block("loop"):
                pos_rest = np.multiply(pos1, DIR1) + np.multiply(pos2, DIR2)
                pos0 = pos_rest + np.multiply(np.mean(RANGEX), DIRX)
                key = tuple(np.round(pos_rest, 4))
                print("========= {} ({}/{})=========".format(key, count, LEN_ALL))
                T_be = SE3(R_be, pos0)
                T_bp = np.matmul(T_be, SE3(np.identity(3), (0.35,0,0.35)))
                gscene.create_safe(GEOTYPE.BOX, "sweep_surf", ROBOT_BASE, 
                                   dims = np.multiply(RANGEX[1]-RANGEX[0], DIRX) + np.multiply(0.01, DIR1) + np.multiply(0.4, DIR2), 
                                   center=T_bp[:3,3], rpy=(0,0,0), 
                                   color=(0.8,0.0,0.0,0.5), display=True,  
                                   fixed=True, collision=True)
                gscene.update_markers_all()
                if VISUALIZE:
                    gscene.add_highlight_axis("hl", "tbe0", link_name=ROBOT_BASE, 
                                              center=T_be[:3,3], orientation_mat=T_be[:3,:3], dims=[0.2,0.04,0.04])
                
                p2p_list = []
                qpair_list = []
                T_be0 = np.copy(T_be)
                for i_try in range(N_try):
                    traj, success = mplan.planner.plan_py(
                        ROBOT_NAME, TIP_LINK, np.concatenate(T2xyzquat(T_be0)), ROBOT_BASE, HOME_POSE, 
                        timeout=TIMEOUT_MP)
                    if not success:
                        continue
                    for i_px, posx in enumerate(STEPS_X):
                        pos_sweep = np.multiply(posx, DIRX)
                        pos = pos_rest + pos_sweep
                        T_be = SE3(R_be, pos)
                        if i_px > 0: # already done for 0
                            if VISUALIZE:
                                gscene.add_highlight_axis("hl", "tbe0", link_name=ROBOT_BASE, 
                                                          center=T_be[:3,3], orientation_mat=T_be[:3,:3])
                            traj, success = mplan.planner.plan_py(
                                ROBOT_NAME, TIP_LINK, np.concatenate(T2xyzquat(T_be)), ROBOT_BASE, HOME_POSE, 
                                timeout=TIMEOUT_MP)
                        if not success:
                            continue
                        Q0 = traj[-1]
                        if VISUALIZE:
                            gscene.show_pose(Q0)

                        pos_tar = np.multiply(RANGEX[0], DIRX)
                        dP_tar = pos_tar - pos_sweep
                        if VISUALIZE:
                            gscene.add_highlight_axis("hl", "tbe_n", link_name=ROBOT_BASE, 
                                                      center=T_be[:3,3]+dP_tar, orientation_mat=T_be[:3,:3])
                        if np.linalg.norm(dP_tar) <= 1e-6:
                            Qdown = Q0
                        else:
                            Traj_down, success = get_sweep_traj(mplan, ee_point, dP_tar, Q0, DP=STEPX, ERROR_CUT=0.01, 
                                                                ref_link=ROBOT_BASE, VISUALIZE=VISUALIZE, VERBOSE=VERBOSE)
                            if len(Traj_down)==0:
                                continue
                            Qdown = Traj_down[-1]

                        pos_tar = np.multiply(RANGEX[1], DIRX)
                        dP_tar = pos_tar - pos_sweep
                        if VISUALIZE:
                            gscene.add_highlight_axis("hl", "tbe_p", link_name=ROBOT_BASE, 
                                                      center=T_be[:3,3]+dP_tar, orientation_mat=T_be[:3,:3])
                        if np.linalg.norm(dP_tar) <= 1e-6:
                            Traj_up = Q0
                        else:
                            Traj_up, success = get_sweep_traj(mplan, ee_point, dP_tar, Q0, DP=STEPX, ERROR_CUT=0.01, 
                                                              ref_link=ROBOT_BASE, VISUALIZE=VISUALIZE, VERBOSE=VERBOSE)
                            if len(Traj_up)==0:
                                continue
                            Qup = Traj_up[-1]

                        p0 = ee_point.get_tf(list2dict(Q0, gscene.joint_names), 
                                             from_link=ROBOT_BASE)[SWEEP_AX,3]
                        pdown = ee_point.get_tf(list2dict(Qdown, gscene.joint_names), 
                                             from_link=ROBOT_BASE)[SWEEP_AX,3]
                        pup = ee_point.get_tf(list2dict(Qup, gscene.joint_names), 
                                             from_link=ROBOT_BASE)[SWEEP_AX,3]
                        p2p = (pdown, pup)
                        p2p0 = (pdown-p0, pup-p0)
                        qpair = (Qdown, Q0, Qup)
                        if np.multiply(*np.sign(p2p0))<0:
                            p2p_list.append(p2p)
                            qpair_list.append(qpair)

                range_list_dict[key] = np.array(p2p_list)
                config_list_dict[key] = np.array(qpair_list)
                if VISUALIZE:
                    gscene.clear_highlight()
        elapsed = gtimer.toc("test")/1000
        print(gtimer)
        print("=============================================")
        print("=============== {} / {} s===================".format(int(elapsed), int(elapsed/count*LEN_ALL)))

        print("=============================================")

    best_range_dict = {}
    for k,v in range_list_dict.items():
        if len(v)>0:
            i_max = np.argmax(v[:,1]-v[:,0])
            best_range_dict[k] = v[i_max]
        else:
            best_range_dict[k] = (0,0)
    np.save(DATASET_PATH, {"best_range_dict": best_range_dict, "range_list_dict": range_list_dict, "config_list_dict": config_list_dict})

### Converting p0 mistaske 1101

In [20]:
# # HOME_POSE = np.zeros(gscene.joint_num)
# HOME_POSE = HOME_POSE_EXT
# for tip_dir, SWEEP_AXIS in [(SweepDirections.front, "Z"), (SweepDirections.front, "X"), (SweepDirections.up, "Z"), (SweepDirections.down, "Z")]:
#     SWEEP_AX = "XYZ".index(SWEEP_AXIS)
#     STEP_SIZES = [0.02]*3
#     STEP_SIZES[SWEEP_AX] = 0.01
#     X_step, Y_step, Z_step = STEP_SIZES

#     if tip_dir == SweepDirections.front:
#         X_range = (0.0, 1)
#         Y_range = (-0.8, 0.8)
#         Z_range = (0.0, 1.0)
#     elif tip_dir == SweepDirections.up:
#         X_range = (0.2, 0.8)
#         Y_range = (-0.8, 0.8)
#         Z_range = (-0.1, 1.3)
#     elif tip_dir == SweepDirections.down:
#         X_range = (0.2, 0.8)
#         Y_range = (-0.8, 0.8)
#         Z_range = (-0.7, 0.7)
#     else:
#         raise(NotImplementedError("SweepDirection not implemented"))
        
#     DATASET_PATH = os.path.join(DATASET_DIR, SweepDirections.get_file_name(ROBOT_TYPE, tip_dir.name+SWEEP_AXIS))
#     R_be, STEP_AX, LEVEL_AX = SweepDirections.get_Re_step_level(tip_dir, SWEEP_AX)
        
#     gtimer = GlobalTimer.instance()
#     gtimer.reset()

#     X_DIR, Y_DIR, Z_DIR = np.identity(3, dtype=np.int)
#     SWEEP_DIR = [X_DIR, Y_DIR, Z_DIR][SWEEP_AX]
#     RANGE_STEPS_REF = [(X_range, X_step, X_DIR), (Y_range, Y_step, Y_DIR), (Z_range, Z_step, Z_DIR)]
#     RANGEX, STEPX, DIRX = RANGE_STEPS_REF[SWEEP_AX]
#     STEP_SWEEP = float(RANGEX[1] - RANGEX[0])/SWEEP_DIV
    
#     RANGE1, STEP1, DIR1 = RANGE_STEPS_REF[LEVEL_AX]
#     RANGE2, STEP2, DIR2 = RANGE_STEPS_REF[STEP_AX]
    
#     filename = SweepDirections.get_file_name(ROBOT_TYPE, tip_dir.name+SWEEP_AXIS)
#     DATASET_PATH = os.path.join(DATASET_DIR, filename+".npy")
#     try:
#         dataset_read = np.reshape(np.load(DATASET_PATH, allow_pickle=True), (1,))[0]
#         print("Loaded: {}".format(filename))
#     except:
#         print("File not exist: {}".format(filename))
#         continue
#     range_list_dict_old = dataset_read['range_list_dict']
#     config_list_dict_old = dataset_read['config_list_dict']
        

#     range_list_dict = {}
#     config_list_dict = {}
#     STEPS_1 = np.round(np.arange(RANGE1[0], RANGE1[1]+STEP1, STEP1), 4)
#     STEPS_2 = np.round(np.arange(RANGE2[0], RANGE2[1]+STEP2, STEP2), 4)
#     STEPS_X = np.round(np.arange(RANGEX[0], RANGEX[1]-1e-3, STEP_SWEEP)[1:].tolist(), 4)
#     LEN_1, LEN_2 = len(STEPS_1), len(STEPS_2)
#     LEN_ALL = LEN_1*LEN_2
#     for i_p1, pos1 in enumerate(STEPS_1):
#         for i_p2, pos2 in enumerate(STEPS_2):
#             pos_rest = np.multiply(pos1, DIR1) + np.multiply(pos2, DIR2)
#             pos0 = pos_rest + np.multiply(np.mean(RANGEX), DIRX)
#             key = tuple(np.round(pos_rest, 4))
#             key_old = tuple(np.round(pos0, 4))
#             if key_old in range_list_dict_old:
#                 range_list_dict[key] = range_list_dict_old[key_old]
#                 config_list_dict[key] = config_list_dict_old[key_old]
                
#     best_range_dict = {}
#     for k,v in range_list_dict.items():
#         if len(v)>0:
#             i_max = np.argmax(v[:,1]-v[:,0])
#             best_range_dict[k] = v[i_max]
#         else:
#             best_range_dict[k] = (0,0)
#     np.save(DATASET_PATH, {"best_range_dict": best_range_dict, "range_list_dict": range_list_dict, "config_list_dict": config_list_dict})

File not exist: indy7-frontZ
Loaded: indy7-frontX
[ 0.  -0.8  0. ] / [ 0.5 -0.8  0. ]
[ 0.   -0.78  0.  ] / [ 0.5  -0.78  0.  ]
[ 0.   -0.76  0.  ] / [ 0.5  -0.76  0.  ]
[ 0.   -0.74  0.  ] / [ 0.5  -0.74  0.  ]
[ 0.   -0.72  0.  ] / [ 0.5  -0.72  0.  ]
[ 0.  -0.7  0. ] / [ 0.5 -0.7  0. ]
[ 0.   -0.68  0.  ] / [ 0.5  -0.68  0.  ]
[ 0.   -0.66  0.  ] / [ 0.5  -0.66  0.  ]
[ 0.   -0.64  0.  ] / [ 0.5  -0.64  0.  ]
[ 0.   -0.62  0.  ] / [ 0.5  -0.62  0.  ]
[ 0.  -0.6  0. ] / [ 0.5 -0.6  0. ]
[ 0.   -0.58  0.  ] / [ 0.5  -0.58  0.  ]
[ 0.   -0.56  0.  ] / [ 0.5  -0.56  0.  ]
[ 0.   -0.54  0.  ] / [ 0.5  -0.54  0.  ]
[ 0.   -0.52  0.  ] / [ 0.5  -0.52  0.  ]
[ 0.  -0.5  0. ] / [ 0.5 -0.5  0. ]
[ 0.   -0.48  0.  ] / [ 0.5  -0.48  0.  ]
[ 0.   -0.46  0.  ] / [ 0.5  -0.46  0.  ]
[ 0.   -0.44  0.  ] / [ 0.5  -0.44  0.  ]
[ 0.   -0.42  0.  ] / [ 0.5  -0.42  0.  ]
[ 0.  -0.4  0. ] / [ 0.5 -0.4  0. ]
[ 0.   -0.38  0.  ] / [ 0.5  -0.38  0.  ]
[ 0.   -0.36  0.  ] / [ 0.5  -0.36  0.  ]
[ 0.   -0.34  

[0.   0.46 0.16] / [0.5  0.46 0.16]
[0.   0.48 0.16] / [0.5  0.48 0.16]
[0.   0.5  0.16] / [0.5  0.5  0.16]
[0.   0.52 0.16] / [0.5  0.52 0.16]
[0.   0.54 0.16] / [0.5  0.54 0.16]
[0.   0.56 0.16] / [0.5  0.56 0.16]
[0.   0.58 0.16] / [0.5  0.58 0.16]
[0.   0.6  0.16] / [0.5  0.6  0.16]
[0.   0.62 0.16] / [0.5  0.62 0.16]
[0.   0.64 0.16] / [0.5  0.64 0.16]
[0.   0.66 0.16] / [0.5  0.66 0.16]
[0.   0.68 0.16] / [0.5  0.68 0.16]
[0.   0.7  0.16] / [0.5  0.7  0.16]
[0.   0.72 0.16] / [0.5  0.72 0.16]
[0.   0.74 0.16] / [0.5  0.74 0.16]
[0.   0.76 0.16] / [0.5  0.76 0.16]
[0.   0.78 0.16] / [0.5  0.78 0.16]
[0.   0.8  0.16] / [0.5  0.8  0.16]
[ 0.   -0.8   0.18] / [ 0.5  -0.8   0.18]
[ 0.   -0.78  0.18] / [ 0.5  -0.78  0.18]
[ 0.   -0.76  0.18] / [ 0.5  -0.76  0.18]
[ 0.   -0.74  0.18] / [ 0.5  -0.74  0.18]
[ 0.   -0.72  0.18] / [ 0.5  -0.72  0.18]
[ 0.   -0.7   0.18] / [ 0.5  -0.7   0.18]
[ 0.   -0.68  0.18] / [ 0.5  -0.68  0.18]
[ 0.   -0.66  0.18] / [ 0.5  -0.66  0.18]
[ 0.   -0.64  0.

[ 0.   -0.22  0.38] / [ 0.5  -0.22  0.38]
[ 0.   -0.2   0.38] / [ 0.5  -0.2   0.38]
[ 0.   -0.18  0.38] / [ 0.5  -0.18  0.38]
[ 0.   -0.16  0.38] / [ 0.5  -0.16  0.38]
[ 0.   -0.14  0.38] / [ 0.5  -0.14  0.38]
[ 0.   -0.12  0.38] / [ 0.5  -0.12  0.38]
[ 0.   -0.1   0.38] / [ 0.5  -0.1   0.38]
[ 0.   -0.08  0.38] / [ 0.5  -0.08  0.38]
[ 0.   -0.06  0.38] / [ 0.5  -0.06  0.38]
[ 0.   -0.04  0.38] / [ 0.5  -0.04  0.38]
[ 0.   -0.02  0.38] / [ 0.5  -0.02  0.38]
[0.   0.   0.38] / [0.5  0.   0.38]
[0.   0.02 0.38] / [0.5  0.02 0.38]
[0.   0.04 0.38] / [0.5  0.04 0.38]
[0.   0.06 0.38] / [0.5  0.06 0.38]
[0.   0.08 0.38] / [0.5  0.08 0.38]
[0.   0.1  0.38] / [0.5  0.1  0.38]
[0.   0.12 0.38] / [0.5  0.12 0.38]
[0.   0.14 0.38] / [0.5  0.14 0.38]
[0.   0.16 0.38] / [0.5  0.16 0.38]
[0.   0.18 0.38] / [0.5  0.18 0.38]
[0.   0.2  0.38] / [0.5  0.2  0.38]
[0.   0.22 0.38] / [0.5  0.22 0.38]
[0.   0.24 0.38] / [0.5  0.24 0.38]
[0.   0.26 0.38] / [0.5  0.26 0.38]
[0.   0.28 0.38] / [0.5  0.28 0.38

[0.   0.62 0.58] / [0.5  0.62 0.58]
[0.   0.64 0.58] / [0.5  0.64 0.58]
[0.   0.66 0.58] / [0.5  0.66 0.58]
[0.   0.68 0.58] / [0.5  0.68 0.58]
[0.   0.7  0.58] / [0.5  0.7  0.58]
[0.   0.72 0.58] / [0.5  0.72 0.58]
[0.   0.74 0.58] / [0.5  0.74 0.58]
[0.   0.76 0.58] / [0.5  0.76 0.58]
[0.   0.78 0.58] / [0.5  0.78 0.58]
[0.   0.8  0.58] / [0.5  0.8  0.58]
[ 0.  -0.8  0.6] / [ 0.5 -0.8  0.6]
[ 0.   -0.78  0.6 ] / [ 0.5  -0.78  0.6 ]
[ 0.   -0.76  0.6 ] / [ 0.5  -0.76  0.6 ]
[ 0.   -0.74  0.6 ] / [ 0.5  -0.74  0.6 ]
[ 0.   -0.72  0.6 ] / [ 0.5  -0.72  0.6 ]
[ 0.  -0.7  0.6] / [ 0.5 -0.7  0.6]
[ 0.   -0.68  0.6 ] / [ 0.5  -0.68  0.6 ]
[ 0.   -0.66  0.6 ] / [ 0.5  -0.66  0.6 ]
[ 0.   -0.64  0.6 ] / [ 0.5  -0.64  0.6 ]
[ 0.   -0.62  0.6 ] / [ 0.5  -0.62  0.6 ]
[ 0.  -0.6  0.6] / [ 0.5 -0.6  0.6]
[ 0.   -0.58  0.6 ] / [ 0.5  -0.58  0.6 ]
[ 0.   -0.56  0.6 ] / [ 0.5  -0.56  0.6 ]
[ 0.   -0.54  0.6 ] / [ 0.5  -0.54  0.6 ]
[ 0.   -0.52  0.6 ] / [ 0.5  -0.52  0.6 ]
[ 0.  -0.5  0.6] / [ 0.5 -0.

[0.   0.8  0.78] / [0.5  0.8  0.78]
[ 0.  -0.8  0.8] / [ 0.5 -0.8  0.8]
[ 0.   -0.78  0.8 ] / [ 0.5  -0.78  0.8 ]
[ 0.   -0.76  0.8 ] / [ 0.5  -0.76  0.8 ]
[ 0.   -0.74  0.8 ] / [ 0.5  -0.74  0.8 ]
[ 0.   -0.72  0.8 ] / [ 0.5  -0.72  0.8 ]
[ 0.  -0.7  0.8] / [ 0.5 -0.7  0.8]
[ 0.   -0.68  0.8 ] / [ 0.5  -0.68  0.8 ]
[ 0.   -0.66  0.8 ] / [ 0.5  -0.66  0.8 ]
[ 0.   -0.64  0.8 ] / [ 0.5  -0.64  0.8 ]
[ 0.   -0.62  0.8 ] / [ 0.5  -0.62  0.8 ]
[ 0.  -0.6  0.8] / [ 0.5 -0.6  0.8]
[ 0.   -0.58  0.8 ] / [ 0.5  -0.58  0.8 ]
[ 0.   -0.56  0.8 ] / [ 0.5  -0.56  0.8 ]
[ 0.   -0.54  0.8 ] / [ 0.5  -0.54  0.8 ]
[ 0.   -0.52  0.8 ] / [ 0.5  -0.52  0.8 ]
[ 0.  -0.5  0.8] / [ 0.5 -0.5  0.8]
[ 0.   -0.48  0.8 ] / [ 0.5  -0.48  0.8 ]
[ 0.   -0.46  0.8 ] / [ 0.5  -0.46  0.8 ]
[ 0.   -0.44  0.8 ] / [ 0.5  -0.44  0.8 ]
[ 0.   -0.42  0.8 ] / [ 0.5  -0.42  0.8 ]
[ 0.  -0.4  0.8] / [ 0.5 -0.4  0.8]
[ 0.   -0.38  0.8 ] / [ 0.5  -0.38  0.8 ]
[ 0.   -0.36  0.8 ] / [ 0.5  -0.36  0.8 ]
[ 0.   -0.34  0.8 ] / [ 0.5 

...

[0.68 0.04 0.  ] / [0.68 0.04 0.6 ]
[0.68 0.06 0.  ] / [0.68 0.06 0.6 ]
[0.68 0.08 0.  ] / [0.68 0.08 0.6 ]
[0.68 0.1  0.  ] / [0.68 0.1  0.6 ]
[0.68 0.12 0.  ] / [0.68 0.12 0.6 ]
[0.68 0.14 0.  ] / [0.68 0.14 0.6 ]
[0.68 0.16 0.  ] / [0.68 0.16 0.6 ]
[0.68 0.18 0.  ] / [0.68 0.18 0.6 ]
[0.68 0.2  0.  ] / [0.68 0.2  0.6 ]
[0.68 0.22 0.  ] / [0.68 0.22 0.6 ]
[0.68 0.24 0.  ] / [0.68 0.24 0.6 ]
[0.68 0.26 0.  ] / [0.68 0.26 0.6 ]
[0.68 0.28 0.  ] / [0.68 0.28 0.6 ]
[0.68 0.3  0.  ] / [0.68 0.3  0.6 ]
[0.68 0.32 0.  ] / [0.68 0.32 0.6 ]
[0.68 0.34 0.  ] / [0.68 0.34 0.6 ]
[0.68 0.36 0.  ] / [0.68 0.36 0.6 ]
[0.68 0.38 0.  ] / [0.68 0.38 0.6 ]
[0.68 0.4  0.  ] / [0.68 0.4  0.6 ]
[0.68 0.42 0.  ] / [0.68 0.42 0.6 ]
[0.68 0.44 0.  ] / [0.68 0.44 0.6 ]
[0.68 0.46 0.  ] / [0.68 0.46 0.6 ]
[0.68 0.48 0.  ] / [0.68 0.48 0.6 ]
[0.68 0.5  0.  ] / [0.68 0.5  0.6 ]
[0.68 0.52 0.  ] / [0.68 0.52 0.6 ]
[0.68 0.54 0.  ] / [0.68 0.54 0.6 ]
[0.68 0.56 0.  ] / [0.68 0.56 0.6 ]
[0.68 0.58 0.  ] / [0.68 0.5

Loaded: indy7-downZ
[ 0.2 -0.8  0. ] / [ 0.2 -0.8  0. ]
[ 0.2  -0.78  0.  ] / [ 0.2  -0.78  0.  ]
[ 0.2  -0.76  0.  ] / [ 0.2  -0.76  0.  ]
[ 0.2  -0.74  0.  ] / [ 0.2  -0.74  0.  ]
[ 0.2  -0.72  0.  ] / [ 0.2  -0.72  0.  ]
[ 0.2 -0.7  0. ] / [ 0.2 -0.7  0. ]
[ 0.2  -0.68  0.  ] / [ 0.2  -0.68  0.  ]
[ 0.2  -0.66  0.  ] / [ 0.2  -0.66  0.  ]
[ 0.2  -0.64  0.  ] / [ 0.2  -0.64  0.  ]
[ 0.2  -0.62  0.  ] / [ 0.2  -0.62  0.  ]
[ 0.2 -0.6  0. ] / [ 0.2 -0.6  0. ]
[ 0.2  -0.58  0.  ] / [ 0.2  -0.58  0.  ]
[ 0.2  -0.56  0.  ] / [ 0.2  -0.56  0.  ]
[ 0.2  -0.54  0.  ] / [ 0.2  -0.54  0.  ]
[ 0.2  -0.52  0.  ] / [ 0.2  -0.52  0.  ]
[ 0.2 -0.5  0. ] / [ 0.2 -0.5  0. ]
[ 0.2  -0.48  0.  ] / [ 0.2  -0.48  0.  ]
[ 0.2  -0.46  0.  ] / [ 0.2  -0.46  0.  ]
[ 0.2  -0.44  0.  ] / [ 0.2  -0.44  0.  ]
[ 0.2  -0.42  0.  ] / [ 0.2  -0.42  0.  ]
[ 0.2 -0.4  0. ] / [ 0.2 -0.4  0. ]
[ 0.2  -0.38  0.  ] / [ 0.2  -0.38  0.  ]
[ 0.2  -0.36  0.  ] / [ 0.2  -0.36  0.  ]
[ 0.2  -0.34  0.  ] / [ 0.2  -0.34  0.  ]
[ 

[ 0.4  -0.22  0.  ] / [ 0.4  -0.22  0.  ]
[ 0.4 -0.2  0. ] / [ 0.4 -0.2  0. ]
[ 0.4  -0.18  0.  ] / [ 0.4  -0.18  0.  ]
[ 0.4  -0.16  0.  ] / [ 0.4  -0.16  0.  ]
[ 0.4  -0.14  0.  ] / [ 0.4  -0.14  0.  ]
[ 0.4  -0.12  0.  ] / [ 0.4  -0.12  0.  ]
[ 0.4 -0.1  0. ] / [ 0.4 -0.1  0. ]
[ 0.4  -0.08  0.  ] / [ 0.4  -0.08  0.  ]
[ 0.4  -0.06  0.  ] / [ 0.4  -0.06  0.  ]
[ 0.4  -0.04  0.  ] / [ 0.4  -0.04  0.  ]
[ 0.4  -0.02  0.  ] / [ 0.4  -0.02  0.  ]
[0.4 0.  0. ] / [0.4 0.  0. ]
[0.4  0.02 0.  ] / [0.4  0.02 0.  ]
[0.4  0.04 0.  ] / [0.4  0.04 0.  ]
[0.4  0.06 0.  ] / [0.4  0.06 0.  ]
[0.4  0.08 0.  ] / [0.4  0.08 0.  ]
[0.4 0.1 0. ] / [0.4 0.1 0. ]
[0.4  0.12 0.  ] / [0.4  0.12 0.  ]
[0.4  0.14 0.  ] / [0.4  0.14 0.  ]
[0.4  0.16 0.  ] / [0.4  0.16 0.  ]
[0.4  0.18 0.  ] / [0.4  0.18 0.  ]
[0.4 0.2 0. ] / [0.4 0.2 0. ]
[0.4  0.22 0.  ] / [0.4  0.22 0.  ]
[0.4  0.24 0.  ] / [0.4  0.24 0.  ]
[0.4  0.26 0.  ] / [0.4  0.26 0.  ]
[0.4  0.28 0.  ] / [0.4  0.28 0.  ]
[0.4 0.3 0. ] / [0.4 0.3 0. 

[0.6 0.3 0. ] / [0.6 0.3 0. ]
[0.6  0.32 0.  ] / [0.6  0.32 0.  ]
[0.6  0.34 0.  ] / [0.6  0.34 0.  ]
[0.6  0.36 0.  ] / [0.6  0.36 0.  ]
[0.6  0.38 0.  ] / [0.6  0.38 0.  ]
[0.6 0.4 0. ] / [0.6 0.4 0. ]
[0.6  0.42 0.  ] / [0.6  0.42 0.  ]
[0.6  0.44 0.  ] / [0.6  0.44 0.  ]
[0.6  0.46 0.  ] / [0.6  0.46 0.  ]
[0.6  0.48 0.  ] / [0.6  0.48 0.  ]
[0.6 0.5 0. ] / [0.6 0.5 0. ]
[0.6  0.52 0.  ] / [0.6  0.52 0.  ]
[0.6  0.54 0.  ] / [0.6  0.54 0.  ]
[0.6  0.56 0.  ] / [0.6  0.56 0.  ]
[0.6  0.58 0.  ] / [0.6  0.58 0.  ]
[0.6 0.6 0. ] / [0.6 0.6 0. ]
[0.6  0.62 0.  ] / [0.6  0.62 0.  ]
[0.6  0.64 0.  ] / [0.6  0.64 0.  ]
[0.6  0.66 0.  ] / [0.6  0.66 0.  ]
[0.6  0.68 0.  ] / [0.6  0.68 0.  ]
[0.6 0.7 0. ] / [0.6 0.7 0. ]
[0.6  0.72 0.  ] / [0.6  0.72 0.  ]
[0.6  0.74 0.  ] / [0.6  0.74 0.  ]
[0.6  0.76 0.  ] / [0.6  0.76 0.  ]
[0.6  0.78 0.  ] / [0.6  0.78 0.  ]
[0.6 0.8 0. ] / [0.6 0.8 0. ]
[ 0.62 -0.8   0.  ] / [ 0.62 -0.8   0.  ]
[ 0.62 -0.78  0.  ] / [ 0.62 -0.78  0.  ]
[ 0.62 -0.76  0.

[0.8  0.38 0.  ] / [0.8  0.38 0.  ]
[0.8 0.4 0. ] / [0.8 0.4 0. ]
[0.8  0.42 0.  ] / [0.8  0.42 0.  ]
[0.8  0.44 0.  ] / [0.8  0.44 0.  ]
[0.8  0.46 0.  ] / [0.8  0.46 0.  ]
[0.8  0.48 0.  ] / [0.8  0.48 0.  ]
[0.8 0.5 0. ] / [0.8 0.5 0. ]
[0.8  0.52 0.  ] / [0.8  0.52 0.  ]
[0.8  0.54 0.  ] / [0.8  0.54 0.  ]
[0.8  0.56 0.  ] / [0.8  0.56 0.  ]
[0.8  0.58 0.  ] / [0.8  0.58 0.  ]
[0.8 0.6 0. ] / [0.8 0.6 0. ]
[0.8  0.62 0.  ] / [0.8  0.62 0.  ]
[0.8  0.64 0.  ] / [0.8  0.64 0.  ]
[0.8  0.66 0.  ] / [0.8  0.66 0.  ]
[0.8  0.68 0.  ] / [0.8  0.68 0.  ]
[0.8 0.7 0. ] / [0.8 0.7 0. ]
[0.8  0.72 0.  ] / [0.8  0.72 0.  ]
[0.8  0.74 0.  ] / [0.8  0.74 0.  ]
[0.8  0.76 0.  ] / [0.8  0.76 0.  ]
[0.8  0.78 0.  ] / [0.8  0.78 0.  ]
[0.8 0.8 0. ] / [0.8 0.8 0. ]
[ 0.82 -0.8   0.  ] / [ 0.82 -0.8   0.  ]
[ 0.82 -0.78  0.  ] / [ 0.82 -0.78  0.  ]
[ 0.82 -0.76  0.  ] / [ 0.82 -0.76  0.  ]
[ 0.82 -0.74  0.  ] / [ 0.82 -0.74  0.  ]
[ 0.82 -0.72  0.  ] / [ 0.82 -0.72  0.  ]
[ 0.82 -0.7   0.  ] / [ 0.82

## plot ranges

In [ ]:
plt.figure(figsize=(15,15*LEN_1))
for i_h, val_1 in enumerate(STEPS_1):
    plt.subplot(LEN_1, 1, i_h+1)
    plt.title(val_1)
    for k,v in range_list_dict.items():
        if round(np.dot(k, DIR1), 4)==round(val_1, 4):
            kk = np.dot(k, DIR2)
            for vv in v:
                plt.plot([kk+0.02, kk+0.02], vv,  '-o', linewidth=1)
            plt.plot([kk, kk], best_range_dict[k], '-ks', linewidth=3)
#     plt.axis([-1,1,0,1])
    plt.grid()

## Read and save min/max csv

In [6]:
for tip_dir, SWEEP_AXIS in [(SweepDirections.front, "Z"), (SweepDirections.front, "X"), (SweepDirections.up, "Z"), (SweepDirections.down, "Z")]:
    filename = SweepDirections.get_file_name(ROBOT_TYPE, tip_dir.name+SWEEP_AXIS)
    DATASET_PATH = os.path.join(DATASET_DIR, filename+".npy")
    try:
        dataset_read = np.reshape(np.load(DATASET_PATH, allow_pickle=True), (1,))[0]
    except:
        print("File not exist: {}".format(filename))
        continue
    range_list_dict = dataset_read['range_list_dict']
    config_list_dict = dataset_read['config_list_dict']
    
    best_range_dict = {}
    best_config_dict = {}
    for k,v in range_list_dict.items():
        if len(v)>0:
            i_max = np.argmax(v[:,1]-v[:,0])
            best_range_dict[k] = v[i_max]
            best_config_dict[k] = config_list_dict[k][i_max]
        else:
            best_range_dict[k] = (0,0)
            best_config_dict[k] = None
    
    SWEEP_AXIS = "XYZ".index(SWEEP_AXIS)
    SWEEP_VEC = [0,0,0]
    SWEEP_VEC[SWEEP_AXIS] = 1
    R_be = SweepDirections.get_dcm_re(tip_dir)
    LEVEL_VEC = np.abs(R_be[:3,0]).astype(np.int) # Assum level along tip-link X axis
    STEP_VEC = np.abs(R_be[:3,1]).astype(np.int)  # Assum step along tip-link Y axis
    assert np.all(np.abs(R_be[:3,2]).astype(int) == SWEEP_VEC), "Sweep direction mismatch"
    LEVEL_AXIS = np.where(LEVEL_VEC)[0][0]
    STEP_AXIS = np.where(STEP_VEC)[0][0]
    min_points = []
    max_points = []
    mm_configs = []
    for k,v in best_range_dict.items():
        min_points.append(tuple(np.add(k, np.multiply(v[0], SWEEP_VEC))))
        max_points.append(tuple(np.add(k, np.multiply(v[1], SWEEP_VEC))))
        mm_configs.append(best_config_dict[k])
    min_points = sorted(min_points, key=lambda x: (x[LEVEL_AXIS], x[STEP_AXIS], x[SWEEP_AXIS]))
    max_points = sorted(max_points, key=lambda x: (x[LEVEL_AXIS], x[STEP_AXIS], x[SWEEP_AXIS]))
    for min_p, max_p in zip(min_points, max_points):
        assert min_p[LEVEL_AXIS] == max_p[LEVEL_AXIS], "Level value mismatch"
        assert min_p[STEP_AXIS] == max_p[STEP_AXIS], "Step value mismatch"
    np.savetxt(os.path.join(DATASET_DIR, filename+'-min.csv'), min_points, delimiter=",")
    np.savetxt(os.path.join(DATASET_DIR, filename+'-max.csv'), max_points, delimiter=",")

File not exist: indy7-frontZ


### Save Qset

In [6]:
# HOME_POSE = np.zeros(gscene.joint_num)
HOME_POSE = HOME_POSE_EXT
for tip_dir, SWEEP_AXIS in [(SweepDirections.front, "Z"), (SweepDirections.front, "X"), (SweepDirections.up, "Z"), (SweepDirections.down, "Z")]:
    SWEEP_AX = "XYZ".index(SWEEP_AXIS)
        
    DATASET_PATH = os.path.join(DATASET_DIR, SweepDirections.get_file_name(ROBOT_TYPE, tip_dir.name+SWEEP_AXIS))
    R_re = SweepDirections.get_dcm_re(tip_dir)
    
    filename = SweepDirections.get_file_name(ROBOT_TYPE, tip_dir.name+SWEEP_AXIS)
    DATASET_PATH = os.path.join(DATASET_DIR, filename+".npy")
    try:
        dataset_read = np.reshape(np.load(DATASET_PATH, allow_pickle=True), (1,))[0]
        print("Loaded: {}".format(filename))
    except:
        print("File not exist: {}".format(filename))
        continue
    best_range_dict = dataset_read['best_range_dict']
    range_list_dict = dataset_read['range_list_dict']
    config_list_dict = dataset_read['config_list_dict']
    Pos_Rotvec_Qlist_dict = defaultdict(lambda : defaultdict(list))
    for p0, qset_list in config_list_dict.items():
        range_list = range_list_dict[p0]
        assert len(qset_list) == len(range_list), "mismatch between qset and range list"
        if len(range_list)>0:
            i_max = np.argmax(range_list[:,1]-range_list[:,0])
            qset_best = qset_list[i_max]
        for q in qset_best:
            Tre = ee_point.get_tf(q, from_link=ROBOT_BASE)
            xyz, rotvec = T2xyzrvec(Tre, 2)
            Pos_Rotvec_Qlist_dict[xyz][rotvec].append(q[crob.idx_dict[ROBOT_NAME]])

    Pos_Rotvec_Qlist_dict = {k1: {k2: v2 for k2, v2 in v1.items()} for k1, v1 in Pos_Rotvec_Qlist_dict.items()}
    PRQ_PATH = os.path.join(DATASET_DIR, filename+"-PRQ.pkl")
    save_pickle(PRQ_PATH, Pos_Rotvec_Qlist_dict)
    print("Saved: {}".format(filename+"-PRQ.pkl"))    

File not exist: indy7-frontZ
Loaded: indy7-frontX
Saved: indy7-frontX-PRQ.pkl
Loaded: indy7-upZ
Saved: indy7-upZ-PRQ.pkl
Loaded: indy7-downZ
Saved: indy7-downZ-PRQ.pkl


### Load Qset

In [7]:
from pkg.planning.motion.moveit.moveit_py import PlannerConfig

In [8]:
from pkg.planning.motion.moveit.moveit_py import PlannerConfig
mplan.reset_PRQdict(enable_PRQ=True, radii=5e-2)
for tip_dir, SWEEP_AXIS in [
    (SweepDirections.front, "Z"), (SweepDirections.front, "X"), (SweepDirections.up, "Z"), (SweepDirections.down, "Z")]:
    filename = SweepDirections.get_file_name(ROBOT_TYPE, tip_dir.name+SWEEP_AXIS)+"-PRQ.pkl"
    PRQ_PATH = os.path.join(DATASET_DIR, filename)
    try:
        Pos_Rotvec_Qlist_dict = load_pickle(PRQ_PATH)
        mplan.register_PRQ(ROBOT_NAME, Pos_Rotvec_Qlist_dict, decimal=2)
        print("Loaded: {}".format(filename))
    except:
        print("File not exist: {}".format(filename))
        continue
    

File not exist: indy7-frontZ-PRQ.pkl
Loaded: indy7-frontX-PRQ.pkl
Loaded: indy7-upZ-PRQ.pkl
Loaded: indy7-downZ-PRQ.pkl


## Show saved ranges

In [9]:
tip_dir, SWEEP_AXIS = [
    (SweepDirections.front, "Z"), (SweepDirections.front, "X"), (SweepDirections.up, "Z"), (SweepDirections.down, "Z")][3]
filename = SweepDirections.get_file_name(ROBOT_TYPE, tip_dir.name+SWEEP_AXIS)
DATASET_PATH = os.path.join(DATASET_DIR, filename+".npy")
sweep_path = os.path.join(DATASET_DIR, filename)
sweep_max = np.loadtxt(sweep_path + "-max.csv", delimiter=",")
sweep_min = np.loadtxt(sweep_path + "-min.csv", delimiter=",")
lines = [(p_min, p_max) for (p_min, p_max) in zip(sweep_min, sweep_max)]

from demo_utils.area_select import show_lines
gscene.clear_highlight()
# show_lines(gscene, lines, base_link=ROBOT_BASE, sweep_axis=SWEEP_AXIS.lower())

lines_srt = sorted(lines, key=lambda x: np.linalg.norm(np.subtract(*x)))
print(len(lines_srt))

2592


In [12]:
fail_list = []
fail_traj_list = []
for i_l, line in enumerate(list(reversed(lines_srt))[:2000:100]):
    print("============== IDX: {} ==============".format(i_l))
    time.sleep(0.1)
    gscene.clear_highlight()
    time.sleep(0.1)
    show_lines(gscene, [line], base_link=ROBOT_BASE, sweep_axis=SWEEP_AXIS.lower())
    minP, maxP = sorted(line, key=lambda x: (np.linalg.norm(x)))
    delta_P = np.subtract(maxP, minP) * 0.9
    mean_P = np.mean(line, axis=0)
    P1 = mean_P + delta_P/2
    if np.linalg.norm(delta_P) < 1e-5:
        continue
    HOME_POSE = crob.home_pose
    R_be = SweepDirections.get_dcm_re(tip_dir)
    T_be = SE3(R_be, P1)
    try_num = 10
    max_traj = []
    for _ in range(try_num):
#         mplan.planner.plan_py(
#             ROBOT_NAME, TIP_LINK, np.concatenate(T2xyzquat(T_be)), ROBOT_BASE, HOME_POSE, 
#             timeout=TIMEOUT_MP)
        ################### PRQ version copy ########################
        self = mplan
        from_Q = HOME_POSE
        tool = DummyObject()
        tool.geometry = ee_point
        target = DummyObject()
        target.geometry = floor
        T_re = T_be
        T_tar_tool = np.matmul(gscene.get_tf(ROBOT_BASE, HOME_POSE), T_re)
        group_name = ROBOT_NAME
        goal_pose = T2xyzquat(T_tar_tool)
        goal_pose = tuple(goal_pose[0]) + tuple(goal_pose[1])
        timeout = timeout_joint = TIMEOUT_MP
        verbose = True
        kwargs = dict()
        #-----------------------------------------------------------#
        
        from_Q_tmp = np.copy(from_Q)
        replan_joint = False
        if self.enable_PRQ:
            T_rtar = self.gscene.get_tf(target.geometry.link_name, from_Q,
                                        from_link=self.chain_dict[group_name]['link_names'][0])
            Tre = np.matmul(T_rtar, T_tar_tool)
            ####
            assert np.linalg.norm(np.matmul(T_be, SE3_inv(T_re)) - np.identity(4)) <1e-2
            ####
            Q = self.sample_PRQ(group_name, Tre, radii=self.radii)
            if Q is not None:
                from_Q_tmp[self.combined_robot.idx_dict[group_name]] = Q
                if self.validate_trajectory([from_Q_tmp], update_gscene=False):
                    replan_joint = True
                    print("use PRQ")
                else:
                    from_Q_tmp = from_Q

        if verbose:
            print("try transition motion") ## <- DO NOT REMOVE THIS: helps multi-process issue with boost python-cpp
        trajectory, success = self.planner.plan_py(
            group_name, tool.geometry.link_name, goal_pose, target.geometry.link_name, tuple(from_Q_tmp),
            timeout=timeout, **kwargs)

        if success and replan_joint:
            kwargs = copy.deepcopy(kwargs)
            kwargs.update(self.pqr_kwargs)
            to_Q = trajectory[-1][self.combined_robot.idx_dict[group_name]]
            trajectory, success = self.planner.plan_joint_motion_py(
                group_name, tuple(to_Q), tuple(from_Q), timeout=timeout_joint, **kwargs)
        #-----------------------------------------------------------#
        traj = trajectory
        success = success
        ################### PRQ version copy ########################
        
        if not success:
            continue
        Q0 = traj[-1]
        traj_sweep, success = get_sweep_traj(mplan, ee_point, -delta_P, Q0, DP=0.01, ERROR_CUT=0.01, 
                                             ref_link=ROBOT_BASE, VISUALIZE=False, VERBOSE=True)
        max_traj = traj_sweep if len(traj_sweep) > len(max_traj) else max_traj
        if success:
            break
    gscene.show_motion(traj)
    gscene.show_motion(max_traj)
    if not success:
        fail_list.append(line)
        fail_traj_list.append(max_traj)
        print("**************** FAIL: {} **************".format(i_l))

============== IDX: 0 ==============
try transition motion
collision
try transition motion
collision
try transition motion
singular
try transition motion
collision
try transition motion
singular
try transition motion
end
============== IDX: 1 ==============
use PRQ
try transition motion
collision
use PRQ
try transition motion
collision
use PRQ
try transition motion
collision
use PRQ
try transition motion
collision
use PRQ
try transition motion
collision
use PRQ
try transition motion
collision
use PRQ
try transition motion
end
============== IDX: 2 ==============
use PRQ
try transition motion
collision
use PRQ
try transition motion
collision
use PRQ
try transition motion
collision
use PRQ
try transition motion
collision
use PRQ
try transition motion
end
============== IDX: 3 ==============
use PRQ
try transition motion
end
============== IDX: 4 ==============
use PRQ
try transition motion
end
============== IDX: 5 ==============
use PRQ
try transition motion
collision
use PRQ
try transi

In [13]:
print(len(fail_list))
time.sleep(0.1)
gscene.clear_highlight()
time.sleep(0.1)
show_lines(gscene, fail_list, base_link=ROBOT_BASE, sweep_axis=SWEEP_AXIS.lower())

4


In [16]:
gscene.clear_highlight()

In [ ]:
# DATASET_PATH_BAK1 = os.path.join(DATASET_DIR+"_bak", ROBOT_TYPE.name+'-up.npy')
# DATASET_PATH_BAK2 = os.path.join(DATASET_DIR+"_bak2", ROBOT_TYPE.name+'-up.npy')
# dataset_read_bak1 = np.reshape(np.load(DATASET_PATH_BAK1, allow_pickle=True), (1,))[0]
# dataset_read_bak2 = np.reshape(np.load(DATASET_PATH_BAK2, allow_pickle=True), (1,))[0]
# for k, v in dataset_read_bak1.items():
#     v.update(dataset_read_bak2[k])
# np.save(DATASET_PATH, dataset_read_bak1)